# MOVIE RECOMMENDER ENGINE

## Library

In [1]:
%matplotlib inline
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import operator


from IPython.display import Image, HTML, display
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise.model_selection import cross_validate
from surprise import Reader, SVD, Dataset


import warnings; warnings.simplefilter('ignore')

## LOAD DataFrame

In [2]:
md = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
Yhat = np.load('tf_idf_pred_rating.npy')

In [3]:
N = len(md)

In [4]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 26 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

In [5]:
md.head().transpose()[:20]

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"['Animation', 'Comedy', 'Family']","['Adventure', 'Fantasy', 'Family']","['Romance', 'Comedy']","['Comedy', 'Drama', 'Romance']",['Comedy']
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


In [6]:
md['genres'] = md['genres'].apply(ast.literal_eval)

## Demographic Filtering - Weighted Rating

In [7]:
# IMDB Top Movies Chart Fomula : 
# WR = (v/(v+m).R) + (m/(v+m).C)
# v : số  lượng người bình chọn cho 1 bộ phim 
# m : số lượng bình chọn cần thiếu để được xét vào list - Một phim phải 
# có lượng vote > 95% các phim khác trong list
# R : rating  

In [8]:
vote_counts = md[md['vote_count'].notnull()]['vote_count']

In [9]:
vote_averages = md[md['vote_average'].notnull()]['vote_average']

In [10]:
# mean() function can be used to calculate 
# mean/average of a given list of numbers

C = vote_averages.mean()

In [11]:
# rating trung binh tren toan tap dataset
C

6.362227146814405

In [12]:
# Tính phân vị - tìm giá trị m sao cho 95% giá trị trong list bé hơn m
# 5% còn lại lớn hơn nó 
m = vote_counts.quantile(0.95)

In [13]:
m

2085.599999999995

In [14]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) 
            & (md['vote_average'].notnull())]

In [15]:
qualified = qualified[['poster_path','title','year','vote_count','vote_average','genres']]

In [16]:
qualified.shape

(452, 6)

In [17]:
# sumary : một phim cần phải đạt trên 434 votes để  được xét 
#        : điểm rating trung bình là 5.2 
#        : có 2274 phim đạt yêu cầu 

In [18]:
def weighted_rating(x,m,C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(m+v) * R) + (m/(m+v) * C)

In [19]:
qualified['wr'] = qualified.apply(lambda x: weighted_rating(x,m,C), axis=1)

In [20]:
# ascending : tăng dần 

qualified = qualified.sort_values('wr', ascending = False)

In [21]:

HTML(qualified.head(10).to_html(escape = False))

,poster_path,title,year,vote_count,vote_average,genres,wr
284,,The Shawshank Redemption,1994,8358.0,8.5,"[Drama, Crime]",8.073084
6873,,The Dark Knight,2008,12269.0,8.3,"[Drama, Action, Crime, Thriller]",8.018458
2369,,Fight Club,1999,9678.0,8.3,[Drama],7.956447
692,,The Godfather,1972,6024.0,8.5,"[Drama, Crime]",7.950215
266,,Pulp Fiction,1994,8670.0,8.3,"[Thriller, Crime]",7.924250
7526,,Inception,2010,14075.0,8.1,"[Action, Thriller, Science Fiction, Mystery, Adventure]",7.875732
8472,,Interstellar,2014,11187.0,8.1,"[Adventure, Drama, Science Fiction]",7.826934
321,,Forrest Gump,1994,8147.0,8.2,"[Comedy, Drama, Romance]",7.825427
5009,,The Lord of the Rings: The Return of the King,2003,8226.0,8.1,"[Adventure, Fantasy, Action]",7.748522
949,,The Empire Strikes Back,1980,5998.0,8.2,"[Adventure, Action, Science Fiction]",7.725848


In [22]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [23]:
gen_md.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22321 entries, 0 to 9024
Data columns (total 26 columns):
adult                    22321 non-null bool
belongs_to_collection    4553 non-null object
budget                   22321 non-null int64
homepage                 4957 non-null object
id                       22321 non-null int64
imdb_id                  22321 non-null object
original_language        22321 non-null object
original_title           22321 non-null object
overview                 22298 non-null object
popularity               22321 non-null float64
poster_path              22314 non-null object
production_companies     22321 non-null object
production_countries     22321 non-null object
release_date             22321 non-null object
revenue                  22321 non-null float64
runtime                  22321 non-null float64
spoken_languages         22321 non-null object
status                   22317 non-null object
tagline                  17830 non-null object
ti

### Filter By Genre

In [24]:
def build_chart(genre, percentile):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())]
    qualified = qualified[['poster_path','title','year','vote_count','vote_average','genre']]
    
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: weighted_rating(x,m,C), axis=1)
    
    qualified = qualified.sort_values('wr', ascending= False)
    
    return qualified



In [25]:
pop_gen = pd.DataFrame(gen_md['genre'].value_counts()).reset_index()
pop_gen.column = ['genre','movies']

In [26]:
list_genre =  pop_gen['index']
list_genre.shape

(20,)

In [27]:
HTML(build_chart(list_genre.loc[0],0.9).head(10).to_html(escape = False))

,poster_path,title,year,vote_count,vote_average,genre,wr
284,,The Shawshank Redemption,1994,8358,8.5,Drama,8.285582
692,,The Godfather,1972,6024,8.5,Drama,8.212499
6873,,The Dark Knight,2008,12269,8.3,Drama,8.162215
2369,,Fight Club,1999,9678,8.3,Drama,8.128220
321,,Forrest Gump,1994,8147,8.2,Drama,8.008085
8472,,Interstellar,2014,11187,8.1,Drama,7.963518
472,,Schindler's List,1993,4436,8.3,Drama,7.957018
8551,,Whiplash,2014,4376,8.3,Drama,7.953060
7935,,The Intouchables,2011,5410,8.2,Drama,7.923822
1835,,Life Is Beautiful,1997,3643,8.3,Drama,7.896127


## Movie Description Based Recommender

In [28]:
def graph_result(result):
    md_graph = md[['poster_path','title','description','year','vote_count','vote_average']].copy()
    list_result_dataframe = []
     
    for i in result:
         list_result_dataframe.append(md_graph.iloc[i]) 

    list_result_dataframe = pd.DataFrame(list_result_dataframe)                
    return HTML(list_result_dataframe.to_html(escape = False))

### Content-Based By Title And Description Using Sklearn

In [29]:
md.description = md['description'].fillna('')

In [30]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(md['description'])

In [31]:
tfidf_matrix.shape
# có 9099 vector tương ướng với 9099 overview,
# mỗi vector có 268124 chiều 

(9025, 266476)

In [32]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Compute the cosine similarity matrix

In [33]:
cosine_sim[0]

array([1.        , 0.00679441, 0.        , ..., 0.        , 0.        ,
       0.00478805])

In [34]:
cosine_sim[1]

array([0.00679441, 1.        , 0.01528831, ..., 0.        , 0.        ,
       0.00368484])

In [35]:
md = md.reset_index()
titles = md['title']
indices = pd.Series(md.index, index=md['title'])

In [36]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Sharknado 4: The 4th Awakens                          9020
The Last Brickmaker in America                        9021
Rustom                                                9022
Mohenjo Daro                                          9023
The Beatles: Eight Days a Week - The Touring Years    9024
Length: 9025, dtype: int64

In [37]:
md[md['title'] == "The Avengers" ]

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
1707,1707,False,NaN,60000000,[Thriller],NaN,9320,tt0118661,en,The Avengers,...,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Saving the World in Style.,The Avengers,False,4.4,205.0,"British Ministry agent John Steed, under direc...",1998
7839,7839,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[Science Fiction, Action, Adventure]",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,...,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0,When an unexpected enemy emerges and threatens...,2012


In [38]:
def get_recommendations(k,title):
    # Get the index of the movie that matches the title
    idx = []
    idx.append(indices[title])
    result = {}
    
    for i in idx:
                # Get the pairwsie similarity scores of all movies with that movie
                sim_scores = list(enumerate(cosine_sim[i]))

                # Sort the movies based on the similarity scores
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

                # Get the scores of the 10 most similar movies
                #(element number 0 is the movie it self)
                sim_scores = sim_scores[1:11]

                movie_indices = [k[0] for k in sim_scores]
                result[i,title] = movie_indices[:k]
                
    return result    

In [39]:
result = get_recommendations(10,'The Dark Knight')
html = ""

for i in result:
        print(i[0]," - ",i[1])
        display(graph_result(result[i]))
        print("-------------------------------")   

6873  -  The Dark Knight


,poster_path,title,description,year,vote_count,vote_average
7901,,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.The Legend Ends",2012,9263.0,7.6
132,,Batman Forever,"The Dark Knight of Gotham City confronts a dastardly duo: Two-Face and the Riddler. Formerly District Attorney Harvey Dent, Two-Face believes Batman caused the courtroom accident which left him disfigured on one side. And Edward Nygma, computer-genius and former employee of millionaire Bruce Wayne, is out to get the philanthropist; as The Riddler. Former circus acrobat Dick Grayson, his family killed by Two-Face, becomes Wayne's ward and Batman's new partner Robin.Courage now, truth always...",1995,1529.0,5.2
1113,,Batman Returns,"Having defeated the Joker, Batman now faces the Penguin - a warped and deformed individual who is intent on being accepted into Gotham society. Crooked businessman Max Schreck is coerced into helping him become Mayor of Gotham and they both attempt to expose Batman in a different light. Selina Kyle, Max's secretary, is thrown from the top of a building and is transformed into Catwoman - a mysterious figure who has the same personality disorder as Batman. Batman must attempt to clear his name, all the time deciding just what must be done with the Catwoman.The Bat, the Cat, the Penguin.",1992,1706.0,6.6
8196,,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that The Mutants had cast upon his city. Now an old foe wants a reunion and the government wants The Man of Steel to put a stop to Batman.Justice Returns... Vengeance Returns... Redemption Comes to Gotham.,2013,426.0,7.9
7537,,Batman: Under the Red Hood,"Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code.Dare to Look Beneath the Hood.",2010,459.0,7.6
524,,Batman,"The Dark Knight of Gotham City begins his war on crime with his first major enemy being the clownishly homicidal Joker, who has seized control of Gotham's underworld.Have you ever danced with the devil in the pale moonlight?",1989,2145.0,7.0
7871,,Batman: Year One,"Two men come to Gotham City: Bruce Wayne after years abroad feeding his lifelong obsession for justice and Jim Gordon after being too honest a cop with the wrong people elsewhere. After learning painful lessons about the city's corruption on its streets and police department respectively, this pair learn how to fight back their own way. With that, Gotham's evildoers from top to bottom are terrorized by the mysterious Batman and the equally heroic Gordon is assigned to catch him by comrades who both hate and fear him themselves. In the ensuing manhunt, both find much in common as the seeds of an unexpected friendship are laid with additional friends and rivals helping to start the legend.A merciless crime turns a man into an outlaw.",2011,255.0,7.1
2578,,Batman: Mask of the Phantasm,"An old flame of Bruce Wayne's strolls into town, re-heating up the romance between the two. At the same time, a mass murderer with an axe for one hand begins systematically eliminating Gotham's crime bosses. Due to the person's dark appearance, he is mistaken for Batman. Now on the run, Batman must solve the mystery and deal with the romance between him and Andrea Beaumont.The Dark Knight fights to save Gotham city from its deadliest enemy.",1993,218.0,7.4
2695,,JFK,New Orleans District Attorney Jim Garrison discovers there's more to the Kennedy assassination tha

-------------------------------


### Content-Based By Title And Description From Scatch

In [40]:
from ipynb.fs.full.Description_Title_Content_Base_Recommender import cosine_similarity_ver2
cosine_similarity_ver2(11,"The Dark Knight")

array([7901, 8134,  132, 8196, 6125, 2578, 1113,  524, 7871, 7537])

In [41]:
graph_result(list(cosine_similarity_ver2(11,"The Dark Knight")))

,poster_path,title,description,year,vote_count,vote_average
7901,,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.The Legend Ends",2012,9263.0,7.6
8134,,"Batman: The Dark Knight Returns, Part 1","Batman has not been seen for ten years. A new breed of criminal ravages Gotham City, forcing 55-year-old Bruce Wayne back into the cape and cowl. But, does he still have what it takes to fight crime in a new era?Old heroes never die. They just get darker.",2012,410.0,7.7
132,,Batman Forever,"The Dark Knight of Gotham City confronts a dastardly duo: Two-Face and the Riddler. Formerly District Attorney Harvey Dent, Two-Face believes Batman caused the courtroom accident which left him disfigured on one side. And Edward Nygma, computer-genius and former employee of millionaire Bruce Wayne, is out to get the philanthropist; as The Riddler. Former circus acrobat Dick Grayson, his family killed by Two-Face, becomes Wayne's ward and Batman's new partner Robin.Courage now, truth always...",1995,1529.0,5.2
8196,,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that The Mutants had cast upon his city. Now an old foe wants a reunion and the government wants The Man of Steel to put a stop to Batman.Justice Returns... Vengeance Returns... Redemption Comes to Gotham.,2013,426.0,7.9
6125,,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the corruption that plagues his home, Gotham City. Unable to work within the system, he instead creates a new identity, a symbol of fear for the criminal underworld - The Batman.Evil fears the knight.",2005,7511.0,7.5
2578,,Batman: Mask of the Phantasm,"An old flame of Bruce Wayne's strolls into town, re-heating up the romance between the two. At the same time, a mass murderer with an axe for one hand begins systematically eliminating Gotham's crime bosses. Due to the person's dark appearance, he is mistaken for Batman. Now on the run, Batman must solve the mystery and deal with the romance between him and Andrea Beaumont.The Dark Knight fights to save Gotham city from its deadliest enemy.",1993,218.0,7.4
1113,,Batman Returns,"Having defeated the Joker, Batman now faces the Penguin - a warped and deformed individual who is intent on being accepted into Gotham society. Crooked businessman Max Schreck is coerced into helping him become Mayor of Gotham and they both attempt to expose Batman in a different light. Selina Kyle, Max's secretary, is thrown from the top of a building and is transformed into Catwoman - a mysterious figure who has the same personality disorder as Batman. Batman must attempt to clear his name, all the time deciding just what must be done with the Catwoman.The Bat, the Cat, the Penguin.",1992,1706.0,6.6
524,,Batman,"The Dark Knight of Gotham City begins his war on crime with his first major enemy being the clownishly homicidal Joker, who has seized control of Gotham's underworld.Have you ever danced with the devil in the pale moonlight?",1989,2145.0,7.0
7871,,Batman: Year One,"Two men come to Gotham City: Bruce Wayne after years abroad feeding his lifelong obsession for justice and Jim Gordon after being too honest a cop with the wrong people elsewhere. After learning painful lessons about the city's corruption on its streets and police department respectively, this pair learn how to fight back their own way. With that, Gotham's evildoers from top to bottom are terrorized by the mysterious Batman and the equally heroic Gordon is assigned to catch him by comrades who both hate and fear him themselves. In the ensuing 

### User Vector

In [42]:
ratings_train = train[['userId','id','rating','movieIndex']]
ratings_matrix = ratings_train.values

In [43]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [44]:
def get_list_movie_have_not_rated(rate_matrix, userId,movie_count):
    user_rated, score = get_items_rated_by_user(rate_matrix,userId)
    result = []
    for i in range(movie_count):
         if i not in user_rated: result.append(i)
    return (userId, result)            

In [45]:
def get_pred_rating(rate_matrix, userId,movie_count):
    userId, list_not_rate =  get_list_movie_have_not_rated(rate_matrix, userId,movie_count)
    result = {}
    for i in list_not_rate:
        result[i] = Yhat[i, userId]
    
    result = sorted(result.items(), key=operator.itemgetter(1),reverse=True)
    list_movie = []
    for i in result:
        list_movie.append(i[0])
    return list_movie

In [46]:
#result,score = get_items_rated_by_user(ratings_matrix,2)
#display(graph_result(result))

In [47]:
result = get_pred_rating(ratings_matrix,1,N)
graph_result(result[0:10])

,poster_path,title,description,year,vote_count,vote_average
5713,,A Charlie Brown Christmas,"When Charlie Brown complains about the overwhelming materialism that he sees amongst everyone during the Christmas season, Lucy suggests that he become director of the school Christmas pageant. Charlie Brown accepts, but is a frustrating struggle. When an attempt to restore the proper spirit with a forlorn little fir Christmas tree fails, he needs Linus' help to learn the meaning of Christmas.That's what Christmas is all about, Charlie Brown!",1965,156.0,7.5
3030,,The Five Senses,Three sets of tenants in a live/work building have daily lives and/or current stories acutely involving one or more of the five senses.,1999,8.0,6.6
4400,,Black Christmas,"A sorority house is terrorized by a stranger who makes frightening phone calls and then murders the sorority sisters during Christmas break.If this movie doesn’t make your skin crawl, it’s on too tight!",1974,133.0,6.8
6634,,How the Grinch Stole Christmas!,"Bitter and hateful, the Grinch is irritated at the thought of the nearby village having a happy time celebrating Christmas. So disguised as Santa Claus, with his dog made to look like a reindeer, he raids the village to steal all the Christmas things. The village is sure to have a sad Christmas this year.You're a mean one, Mr. Grinch",1966,364.0,7.2
1642,,One Magic Christmas,"Ginny Grainger, a young mother, rediscovers the joy and beauty of Christmas, thanks to the unshakable faith of her six-year-old daughter Abbie and Gideon, Ginny's very own guardian angel.",1985,15.0,5.7
6563,,Black Christmas,"An escaped maniac returns to his childhood home on Christmas Eve, which is now a sorority house, and begins to murder the sorority sisters one by one. A remake of the 1974 horror movie, Black Christmas (1974).This holiday season, the slay ride begins.",2006,119.0,4.8
8955,,A Very Murray Christmas,"Bill Murray worries no one will show up to his TV show due to a massive snowstorm in New York City. Through luck and perseverance, guests arrive at Gotham’s Carlyle Hotel to help him — dancing and singing in holiday spirit.We wish you a Murray Christmas.",2015,149.0,5.2
1639,,The Muppet Christmas Carol,"A retelling of the classic Dickens tale of Ebenezer Scrooge, miser extraordinaire. He is held accountable for his dastardly ways during night-time visitations by the Ghosts of Christmas Past, Present, and future.",1992,217.0,7.2
8196,,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that The Mutants had cast upon his city. Now an old foe wants a reunion and the government wants The Man of Steel to put a stop to Batman.Justice Returns... Vengeance Returns... Redemption Comes to Gotham.,2013,426.0,7.9
7537,,Batman: Under the Red Hood,"Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code.Dare to Look Beneath the Hood.",2010,459.0,7.6


## Metadata Based Recommender

### Merging cast crew keyword data into main dataframe

In [48]:
smd =  pd.read_csv('../the-movies-dataset/movies_metadata_merge_crew_keywords.csv')
graph = smd[['poster_path','title','year','vote_count','vote_average','genres']]

### Using Sklearn Library

In [49]:
from ipynb.fs.full.Cast_Director_Gernes_Content_Base_Recommender import  get_recommendations
result = get_recommendations("The Dark Knight",11)
for i in result:
    list_movie_result = result[i] 
graph_result(list_movie_result)    

,poster_path,title,description,year,vote_count,vote_average
7901,,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.The Legend Ends",2012,9263.0,7.6
6125,,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the corruption that plagues his home, Gotham City. Unable to work within the system, he instead creates a new identity, a symbol of fear for the criminal underworld - The Batman.Evil fears the knight.",2005,7511.0,7.5
6518,,The Prestige,"A mysterious story of two magicians whose intense rivalry leads them on a life-long battle for supremacy -- full of obsession, deceit and jealousy with dangerous and deadly consequences.Are You Watching Closely?",2006,4510.0,8.0
2064,,Following,"A struggling, unemployed young writer takes to following strangers around the streets of London, ostensibly to find inspiration for his new novel.",1998,363.0,7.2
7526,,Inception,"Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: ""inception"", the implantation of another person's idea into a target's subconscious.Your mind is the scene of the crime.",2010,14075.0,8.1
4099,,Insomnia,Two Los Angeles homicide detectives are dispatched to a northern town where the sun doesn't set to investigate the methodical murder of a local teen.A tough cop. A brilliant killer. An unspeakable crime.,2002,1181.0,6.8
3360,,Memento,"Suffering short-term memory loss after a head injury, Leonard Shelby embarks on a grim quest to find the lowlife who murdered his wife in this gritty, complex thriller that packs more knots than a hangman's noose. To carry out his plan, Shelby snaps Polaroids of people and places, jotting down contextual notes on the backs of photos to aid in his search and jog his memory. He even tattoos his own body in a desperate bid to remember.Some memories are best forgotten.",2000,4168.0,8.1
8472,,Interstellar,Interstellar chronicles the adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.Mankind was born on Earth. It was never meant to die here.,2014,11187.0,8.1
7537,,Batman: Under the Red Hood,"Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code.Dare to Look Beneath the Hood.",2010,459.0,7.6
1113,,Batman Returns,"Having defeated the Joker, Batman now faces the Penguin - a warped and deformed individual who is intent on being accepted into Gotham society. Crooked businessman Max Schreck is coerced into helping him become Mayor of Gotham and they both attempt to expose Batman in a different light. Selina Kyle, Max's secretary, is thrown from the top of a building and is transformed into Catwoman - a mysterious figure who has the same personality disorder as Batman. Batman must attempt to clear his name, all the time deciding just what must be done with the Catwoman.The Bat, the Cat, the Penguin.",1992,1706.0,6.6


### From Scatch

In [50]:
from ipynb.fs.full.Cast_Director_Gernes_Content_Base_Recommender import  get_recommendations_ver2

In [51]:
result = get_recommendations_ver2("The Dark Knight",11)
for i in result:
    list_movie_result = result[i] 
graph_result(list_movie_result)    

,poster_path,title,description,year,vote_count,vote_average
7901,,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.The Legend Ends",2012,9263.0,7.6
6125,,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the corruption that plagues his home, Gotham City. Unable to work within the system, he instead creates a new identity, a symbol of fear for the criminal underworld - The Batman.Evil fears the knight.",2005,7511.0,7.5
6518,,The Prestige,"A mysterious story of two magicians whose intense rivalry leads them on a life-long battle for supremacy -- full of obsession, deceit and jealousy with dangerous and deadly consequences.Are You Watching Closely?",2006,4510.0,8.0
2064,,Following,"A struggling, unemployed young writer takes to following strangers around the streets of London, ostensibly to find inspiration for his new novel.",1998,363.0,7.2
7526,,Inception,"Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: ""inception"", the implantation of another person's idea into a target's subconscious.Your mind is the scene of the crime.",2010,14075.0,8.1
4099,,Insomnia,Two Los Angeles homicide detectives are dispatched to a northern town where the sun doesn't set to investigate the methodical murder of a local teen.A tough cop. A brilliant killer. An unspeakable crime.,2002,1181.0,6.8
3360,,Memento,"Suffering short-term memory loss after a head injury, Leonard Shelby embarks on a grim quest to find the lowlife who murdered his wife in this gritty, complex thriller that packs more knots than a hangman's noose. To carry out his plan, Shelby snaps Polaroids of people and places, jotting down contextual notes on the backs of photos to aid in his search and jog his memory. He even tattoos his own body in a desperate bid to remember.Some memories are best forgotten.",2000,4168.0,8.1
8472,,Interstellar,Interstellar chronicles the adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.Mankind was born on Earth. It was never meant to die here.,2014,11187.0,8.1
7537,,Batman: Under the Red Hood,"Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code.Dare to Look Beneath the Hood.",2010,459.0,7.6
1113,,Batman Returns,"Having defeated the Joker, Batman now faces the Penguin - a warped and deformed individual who is intent on being accepted into Gotham society. Crooked businessman Max Schreck is coerced into helping him become Mayor of Gotham and they both attempt to expose Batman in a different light. Selina Kyle, Max's secretary, is thrown from the top of a building and is transformed into Catwoman - a mysterious figure who has the same personality disorder as Batman. Batman must attempt to clear his name, all the time deciding just what must be done with the Catwoman.The Bat, the Cat, the Penguin.",1992,1706.0,6.6


## Collaborative Filtering 